# Pytree 사용하기

JAX는 딕셔너리, 배열, 중첩 리스트, 딕셔너리 리스트 혼합 중첩 등 다양한 구조에 대한 연산을 지원하는데, 이러한 구조들을 JAX에서는 pytree라고 부릅니다. 해당 문서에서는 pytree를 잘 사용하는 법과 실수할 수 있는 부분들을 짚고 넘어가겠습니다.

## pytree란

[공식 도큐먼트](https://jax.readthedocs.io/en/latest/working-with-pytrees.html) 에서 pytree에 대해 설명한 글을 직역하면 다음과 같습니다:

*파이 트리는 컨테이너 형태의 Python 객체들로 구성된 구조입니다. “리프(leaf)” 파이 트리와 더 많은 파이 트리들로 이루어질 수 있습니다. 파이 트리는 리스트, 튜플, 사전을 포함할 수 있습니다. 리프(leaf)는 배열과 같이 파이 트리가 아닌 모든 것을 의미하지만, 단일 리프도 파이 트리입니다.*

(어떻게 더 잘 번역을 해야할지 감이 안와 공식 도큐먼트를 그대로 인용합니다.)

머신러닝의 관점에서 pytree는 다음과 같은 정보를 포함할 수 있습니다:

- 모델 파라메터
- 데이터셋
- 강화학습 문제에서의 에이전트 observation

아래는 간단한 pytree 예제입니다. example_trees 라는 pytree object를 만들어 보겠습니다:

In [3]:
import jax
import jax.numpy as jnp

example_trees = [
    [1, 'a', object()],
    (1, (2, 3), ()),
    [1, {'k1': 2, 'k2': (3, 4)}, 5],
    {'a': 2, 'b': (2, 3)},
    jnp.array([1, 2, 3]),
]

`jax.tree.leaves()`는 리프 노드들을 flatten한 값을 추출합니다. 아래는 `jax.tree.leaves()` 없이 출력한 결과입니다:

In [4]:
for pytree in example_trees:
    print(f"{repr(pytree):<45} has {len(pytree)} leaves: {pytree} leavers: {pytree}")

[1, 'a', <object object at 0x107cbb6c0>]      has 3 leaves: [1, 'a', <object object at 0x107cbb6c0>] leavers: [1, 'a', <object object at 0x107cbb6c0>]
(1, (2, 3), ())                               has 3 leaves: (1, (2, 3), ()) leavers: (1, (2, 3), ())
[1, {'k1': 2, 'k2': (3, 4)}, 5]               has 3 leaves: [1, {'k1': 2, 'k2': (3, 4)}, 5] leavers: [1, {'k1': 2, 'k2': (3, 4)}, 5]
{'a': 2, 'b': (2, 3)}                         has 2 leaves: {'a': 2, 'b': (2, 3)} leavers: {'a': 2, 'b': (2, 3)}
Array([1, 2, 3], dtype=int32)                 has 3 leaves: [1 2 3] leavers: [1 2 3]


아래 코드는 `jax.tree.leaves()`를 사용하여 리프 노드를 출력한 결과입니다:

In [5]:
for pytree in example_trees:
    leaves = jax.tree.leaves(pytree)
    print(f"{repr(pytree):<45} has {len(leaves)} leaves: {leaves} leavers: {leaves}")

[1, 'a', <object object at 0x107cbb6c0>]      has 3 leaves: [1, 'a', <object object at 0x107cbb6c0>] leavers: [1, 'a', <object object at 0x107cbb6c0>]
(1, (2, 3), ())                               has 3 leaves: [1, 2, 3] leavers: [1, 2, 3]
[1, {'k1': 2, 'k2': (3, 4)}, 5]               has 5 leaves: [1, 2, 3, 4, 5] leavers: [1, 2, 3, 4, 5]
{'a': 2, 'b': (2, 3)}                         has 3 leaves: [2, 2, 3] leavers: [2, 2, 3]
Array([1, 2, 3], dtype=int32)                 has 1 leaves: [Array([1, 2, 3], dtype=int32)] leavers: [Array([1, 2, 3], dtype=int32)]
